In [ ]:
def update(self, new_notifica, observed_cols=None):

    if observed_cols is None:
        observed_cols = set(self.df.columns.tolist()).intersection(new_notifica.df.columns.tolist())
        observed_cols.remove('id')
        observed_cols = list(observed_cols)        

    
    novas_notificacoes = new_notifica.df.loc[~new_notifica.df['id'].isin(self.df['id'])].copy().set_index('id')
    
    print(f"novas_notificacoes {len(novas_notificacoes)}")

    possiveis_alterados = self.df.loc[self.df['id'].isin(new_notifica.df['id'])].copy()
    possiveis_atualizacoes = new_notifica.df.loc[new_notifica.df['id'].isin(self.df['id'])].copy()

    mergedf = pd.merge(possiveis_alterados[observed_cols+['id']],possiveis_atualizacoes[observed_cols+['id']],on='id',how='left',suffixes=['_old','_new'])
    mergedf['updated'] = False
    mergedf['updated_cols'] = ''

    for col in observed_cols:
        old_col = f'{col}_old'
        new_col = f'{col}_new'

        criterios = ( ~(mergedf[old_col].isna() & mergedf[new_col].isna()) & (mergedf[old_col]!=mergedf[new_col]) )
        # criterios = ( (mergedf[old_col]!=mergedf[new_col]) )

        mergedf.loc[ criterios, 'updated_cols' ] = mergedf.loc[ criterios,'updated_cols'].apply( lambda l: utils.strlist(l,col) )
        mergedf.loc[ criterios, 'updated' ] = True


    self.df = self.df.set_index('id')
    self.df = self.df.append(novas_notificacoes)
    
    mergedf = mergedf.loc[mergedf['updated']==True]

    updated_cols = mergedf[['updated_cols','id']].groupby('updated_cols').count().sort_values('id',ascending=False).reset_index()
    updated_cols.iloc[0:-1,0] += ','
    updated_cols = list(set(updated_cols.sum()[0].split(',')))

    for col in updated_cols:
        atualizacoes = possiveis_atualizacoes.loc[
            (
                possiveis_atualizacoes['id'].isin(mergedf.loc[mergedf['updated_cols'].str.contains(col),'id'])
            ),
            ['id',col]
        ].copy().set_index('id')

        print(f"atualizacoes {col}: {len(atualizacoes)}")
        self.df.loc[atualizacoes.index,[col]].update(atualizacoes)

    self.df.reset_index(inplace=True)

    mergedf.to_pickle(join(self.database_dir,f"updates_{hoje.strftime('%d_%m_%Y_%H%M')}.pkl"),'bz2')
    return novas_notificacoes, mergedf